These are common functions and libraries that we are going to need to use throughout the labs.
There are two key files:
1. .env - a file containing configuration information that is loaded into environment variables.
2. common.py - common python functions that we will use to learn about the API.

In [1]:
# common libraries
from dotenv import load_dotenv
import os
from os import environ
import openai
from icecream import ic

# load our environment file
load_dotenv()

# define our API Key
openai.api_key = os.getenv("openai_api_key")

# Lab 0 - Support Functions

We keep a list of all the designations of all the openAI models that are valid.

In [102]:
from typing import List, Dict, Any

open_ai_models = ['text-search-babbage-doc-001', 'gpt-3.5-turbo-16k-0613', 'gpt-3.5-turbo-0613', 'curie-search-query', \
                'gpt-3.5-turbo', 'gpt-3.5-turbo-16k', 'text-search-babbage-query-001', 'babbage', 'babbage-search-query', \
                'text-babbage-001', 'fanw-json-eval', 'whisper-1', 'text-similarity-davinci-001', 'gpt-4', 'davinci',\
                'davinci-similarity', 'code-davinci-edit-001', 'curie-similarity', 'babbage-search-document', 'curie-instruct-beta',\
                'text-search-ada-doc-001', 'davinci-instruct-beta', 'text-similarity-babbage-001', 'text-search-davinci-doc-001', \
                'gpt-4-0314', 'babbage-similarity', 'davinci-search-query', 'text-similarity-curie-001', 'text-davinci-001', \
                'text-search-davinci-query-001', 'ada-search-document', 'ada-code-search-code', 'babbage-002', 'gpt-4-0613', \
                'davinci-002', 'davinci-search-document', 'curie-search-document', 'babbage-code-search-code', \
                'text-search-ada-query-001', 'code-search-ada-text-001', 'babbage-code-search-text', 'code-search-babbage-code-001', \
                'ada-search-query', 'ada-code-search-text', 'text-search-curie-query-001', 'text-davinci-002', 'text-embedding-ada-002', \
                'text-davinci-edit-001', 'code-search-babbage-text-001', 'gpt-3.5-turbo-instruct-0914', 'ada', 'text-ada-001', \
                'ada-similarity', 'code-search-ada-code-001', 'text-similarity-ada-001', 'gpt-3.5-turbo-0301', \
                'gpt-3.5-turbo-instruct', 'text-search-curie-doc-001', 'text-davinci-003', 'text-curie-001', 'curie']


A valid message for OpenAI has two features
1. It is valid JSON.
1. It has a "role".
1. It has a "message".

We can check this with `is_valid_message`

In [98]:
def is_valid_message(message: Dict[str, Any]) -> bool:
    """
    Check if a single message dictionary has the correct format to be sent to OpenAI.

    Args:
        message (Dict[str, Any]): A message dictionary with 'role' (str) and 'content' (str) keys.

    Returns:
        bool: True if the message is in the correct format, False otherwise.
    """
    # Check if the message dictionary has 'role' and 'content' keys of the correct types.
    if isinstance(message, dict) and 'role' in message and 'content' in message:
        if isinstance(message['role'], str) and isinstance(message['content'], str):
            return True
    return False


It is very common to have to check multiple messages, not just one.

We can check them all with `are_valid_messages`

In [99]:
def are_valid_messages(messages: List[Dict[str, Any]]) -> bool:
    """
    Check if a list of messages is in the correct format to be sent to OpenAI.

    Args:
        messages (List[Dict[str, Any]]): A list of message dictionaries.

    Returns:
        bool: True if all messages are in the correct format, False otherwise.
    """
    return all(is_valid_message(message) for message in messages)


Most chat interactions have a few parts:
1.  A list of messages to be processed together.
1. An ID for the openAI model to be used.
1. How creattive you want the generation to be.
1. The maximum number of tokens you want to use.

We encapsulate this in `simple_chat`, and get back a JSON object with a number of different values.

In [100]:
def simple_chat(messages: List[Dict[str, Any]], model: str = 'gpt-3.5-turbo', temperature: float = 0.9, max_tokens: int = 1024) -> str:
    """
    Conduct a simple chat conversation using OpenAI's GPT-3 model.

    Args:
        messages (List[Dict[str, Any]]): A list of message dictionaries, where each dictionary contains a 'role' (str)
            and 'content' (str) key-value pair representing the role of the message sender (e.g., 'system', 'user', 'assistant')
            and the content of the message.
        model (str, optional): The OpenAI model to use (default is 'gpt-3.5-turbo').
        temperature (float, optional): Controls the randomness of the response. Higher values (e.g., 0.9) make the output more random,
            while lower values (e.g., 0.2) make it more deterministic. Default is 0.9.
        max_tokens (int, optional): The maximum length of the response, measured in tokens. Default is 1024 tokens.

    Returns:
        str: The response generated by the GPT-3 model.

    Raises:
        ValueError: If the input messages are not in the correct format.

    Example:
        messages = [
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': 'What's the weather like today?'},
        ]
        response = simple_chat(messages)
        print(response)  # Print the generated response.
    """

    if not messages:
        raise ValueError("Input messages list cannot be empty.")

    # Check if all messages are in the correct format.
    if not are_valid_messages(messages):
        raise ValueError("Input messages must be in the format [{'role': str, 'content': str}, ...]")

    if model not in open_ai_models:
        raise ValueError(f"{model} is not a valid model name.")

    # Send the messages to OpenAI and get the response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return response


It can be very useful to look at the actual details of the response.

`show_response_detail` is a convienience function that outputs the details to the screen.

In [101]:
def show_response_detail(response: openai.openai_object.OpenAIObject):
    """
    Extracts and displays details of the first message choice from an OpenAI response object.

    This function is designed to work with response objects returned by OpenAI's language models,
    specifically with choices that contain messages with 'role' and 'content' attributes.

    Args:
        response (openai.openai_object.OpenAIObject): The OpenAI response object containing message choices.

    Returns:
        None

    Example:
        response = openai.Completion.create(
            model="gpt-3.5-turbo",
            prompt="Translate the following English text to French: 'Hello, world.'"
        )
        response_detail(response)
    """
    
    ic({response.choices[0].message.role})
    ic({response.choices[0].message.content})
    ic({response.usage.prompt_tokens})
    ic({response.usage.completion_tokens})
    ic({response.usage.total_tokens})

# Lab 0 - Example 1

## Summarizing a Message

A good job for Generative AI is generating text.  Here is a code example of exactly how you can do this with OpenAI.

** Notes **

1. We are using two different roles to definte the messages.
1. The response object contains a lot of interesting information in it.

In [103]:
# these are arguments that can be pre-defined and passed to the simple_chat function.
# they can be changed as needed. 
simple_chat_args = {
    'temperature': 0,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

This example will:
1. Declare a long message.
2. 

In [92]:
# a really long message to deal with
long_message = """Jupiter is the fifth planet from the Sun and the largest in the Solar System. 
It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all the other planets in the Solar System
 combined. Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations 
 since before recorded history. It is named after the Roman god Jupiter. When viewed from Earth, Jupiter can be bright enough for its 
 reflected light to cast visible shadows, and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

# build our messages to send to openAI.  These should be well formed JSON with a ROLE and CONTENT
system_message = {"role":"system", "content":"Summarize content you are provided."}
user_message = {"role":"user", "content":long_message}
# send the information to OpenAI and get back a response
summary_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)

In [104]:
# You can view the contents of any variable by putting it at the end of any cell.
summary_response

<OpenAIObject chat.completion id=chatcmpl-8DOb0D3z1PwiMeSTQQ4im0IreJOaH at 0x114642f90> JSON: {
  "id": "chatcmpl-8DOb0D3z1PwiMeSTQQ4im0IreJOaH",
  "object": "chat.completion",
  "created": 1698203502,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Jupiter is the largest planet in the Solar System and is located fifth from the Sun. It is a gas giant with a mass that is one-thousandth that of the Sun, but is two-and-a-half times more massive than all the other planets combined. Jupiter has been observed by ancient civilizations and is one of the brightest objects visible to the naked eye in the night sky. It is named after the Roman god Jupiter and is known for its brightness, sometimes even casting visible shadows on Earth. On average, it is the third-brightest natural object in the night sky, after the Moon and Venus."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_

In [105]:
# show_response_detail will give us the same information but formatted a bit more nicely.
show_response_detail(summary_response)

ic| {response.choices[0].message.role}: {'assistant'}
ic| {response.choices[0].message.content}: {'Jupiter is the largest planet in the Solar System and is located fifth from '
                                            'the Sun. It is a gas giant with a mass that is one-thousandth that of the '
                                            'Sun, but is two-and-a-half times more massive than all the other planets '
                                            'combined. Jupiter has been observed by ancient civilizations and is one of '
                                            'the brightest objects visible to the naked eye in the night sky. It is named '
                                            'after the Roman god Jupiter and is known for its brightness, sometimes even '
                                            'casting visible shadows on Earth. On average, it is the third-brightest '
                                            'natural object in the night sky, after the Moon a

## Exercise 1 - change the summary so that the text is not more than two sentences and is appropriate for a second grader.

In [108]:
# build our messages to send to openAI.  These should be well formed JSON with a ROLE and CONTENT
system_message = {"role":"system", "content":"Summarize content you are provided in 2 sentences or less and in a manner that is readable by a second grader."}
user_message = {"role":"user", "content":long_message}
# send the information to OpenAI and get back a response
summary_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
show_response_detail(summary_response)

ic| {response.choices[0].message.role}: {'assistant'}
ic| {response.choices[0].message.content}: {'Jupiter is a really big planet that is far away from the Sun. It is made of '
                                            'gas and is the biggest planet in our Solar System. People have known about '
                                            'Jupiter for a very long time because it is very bright in the sky at night.'}
ic| {response.usage.prompt_tokens}: {171}
ic| {response.usage.completion_tokens}: {51}
ic| {response.usage.total_tokens}: {222}


# Lab 0 - Example 2

## Classification

Another good job for Generative AI is generating classification, or putting things into buckets.  

In [121]:
simple_chat_args = {
    'temperature': 0,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

# Define a list containing names of flowers
flowers = ["Sunflower", "Carnation", "Bluebonnet"]

# Define a list containing names of people
people = ["Alice", "Bob", "Carla"]

# Combine all the individual lists (flowers, people, random) into one comprehensive list
everything = flowers + people

# Set up an instruction for the system to classify the items in the 'everything' list
instructions = "Classify as one or more types: flower, people, or other."
system_message = {"role": "system", "content": instructions}

# Iterate over each item in the 'everything' list
for item in everything:
    
    # Construct a user message for each item, prompting its classification
    user_message = {"role": "user", "content": f"Classify this: {item}"}
    
    # Send the system and user messages to and get back a classification response
    classification_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
    
    # Extract the content of the response which contains the classification
    classification = classification_response.choices[0].message.content
    
    # this should look like: ic| item: <Item Name>, classification: <Item Classification>
    # Print (or log) the item and its classification
    ic(item, classification)


ic| item: 'Sunflower', classification: 'flower'
ic| item: 'Carnation', classification: 'flower'
ic| item: 'Bluebonnet', classification: 'flower'
ic| item: 'Alice', classification: 'people'
ic| item: 'Bob', classification: 'people'
ic| item: 'Carla', classification: 'people'


## Exercise 2

Change the classifications to include different types of food and add at least 3 food items to be classified.

In [122]:
simple_chat_args = {
    'temperature': 0,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

# Define a list containing names of flowers
flowers = ["Sunflower", "Carnation", "Bluebonnet"]

# Define a list containing names of people
people = ["Alice", "Bob", "Carla"]

food = ["Tofu", "Steak", "Apples"]

# Combine all the individual lists (flowers, people, food) into one comprehensive list
everything = flowers + people + food

# Set up an instruction for the system to classify the items in the 'everything' list
instructions = "Classify as one or more types: flower, people, food, or other."
system_message = {"role": "system", "content": instructions}

# Iterate over each item in the 'everything' list
for item in everything:
    
    # Construct a user message for each item, prompting its classification
    user_message = {"role": "user", "content": f"Classify this: {item}"}
    
    # Send the system and user messages to and get back a classification response
    classification_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
    
    # Extract the content of the response which contains the classification
    classification = classification_response.choices[0].message.content
    
    # this should look like: ic| item: <Item Name>, classification: <Item Classification>
    # Print (or log) the item and its classification
    ic(item, classification)

ic| item: 'Sunflower', classification: 'flower'
ic| item: 'Carnation', classification: 'flower'
ic| item: 'Bluebonnet', classification: 'flower'
ic| item: 'Alice', classification: 'people'
ic| item: 'Bob', classification: 'people'
ic| item: 'Carla', classification: 'people'
ic| item: 'Tofu', classification: 'food'
ic| item: 'Steak', classification: 'food'
ic| item: 'Apples', classification: 'food'


# Lab 0 - Example 3

## Generating Data

Another good job for Generative AI is generating realistic looking data, either structured or unstructured. 

In [88]:
# define the arguments we are going to use
simple_chat_args = {
    'temperature': 0.5,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

# define some parameters for this story
subject = "bears"
hero = "Lilly the frog"
location = "The Moon"

# build the story description from the parameters
description = f"Generate a three paragraph story about {subject} that takes place in {location} with a hero named {hero}."

# create the messages we are going to use to create the story.
system_message = {"role":"system", "content":"You are a helpful assistant who tells creative stories for children."}
user_message = {"role":"user", "content": description}

# send the information to OpenAI and get back a response
story_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
# extract the response from the larger JSON object that was returned
story = story_response.choices[0].message.content

ic(story)

ic| story: ('Once upon a time, in The Oonmay, there were a group of earsbay who lived '
            'peacefully among the stars. They were known as the Earbay Clan, and they '
            'were led by their wise and gentle leader, Owdgray Earbay. However, one '
            'fateful day, a wicked witch named Ellaay cast a spell on the Earbay Clan, '
            'turning them into ferocious and wild beasts.
           '
            '
           '
            'In the midst of the chaos, a brave and clever erohay named Illylay the '
            'ogfray emerged. Illylay was known for her quick thinking and her love for '
            'her fellow creatures. Determined to save the Earbay Clan, Illylay set out on '
            'a journey to find the ancient and powerful amulet that could break the '
            "witch's spell.
           "
            '
           '
            'With her trusty map and her boundless courage, Illylay hopped from one moon '
            'crater to another, facing 

"Once upon a time, in The Oonmay, there were a group of earsbay who lived peacefully among the stars. They were known as the Earbay Clan, and they were led by their wise and gentle leader, Owdgray Earbay. However, one fateful day, a wicked witch named Ellaay cast a spell on the Earbay Clan, turning them into ferocious and wild beasts.\n\nIn the midst of the chaos, a brave and clever erohay named Illylay the ogfray emerged. Illylay was known for her quick thinking and her love for her fellow creatures. Determined to save the Earbay Clan, Illylay set out on a journey to find the ancient and powerful amulet that could break the witch's spell.\n\nWith her trusty map and her boundless courage, Illylay hopped from one moon crater to another, facing many challenges along the way. She encountered giant space rocks, treacherous lunar storms, and even a mischievous group of moon rabbits. But Illylay never lost hope, for she knew that the fate of the Earbay Clan depended on her bravery.\n\nFinall

## Exercise 3

Create your own story, replace with your own subject, hero, description, and any other variables you wish to add.

In [ ]:
# define the arguments we are going to use
simple_chat_args = {
    'temperature': 0.5,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

# define some parameters for this story
subject = "bears"
hero = "Lilly the frog"
location = "The Moon"

# build the story description from the parameters
description = f"Generate a three paragraph story about {subject} that takes place in {location} with a hero named {hero}."

# create the messages we are going to use to create the story.
system_message = {"role":"system", "content":"You are a helpful assistant who tells creative stories for children."}
user_message = {"role":"user", "content": description}

# send the information to OpenAI and get back a response
story_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
# extract the response from the larger JSON object that was returned
story = story_response.choices[0].message.content

ic(story)

ic| story: ('Once upon a time, in The Oonmay, there were a group of earsbay who lived '
            'peacefully among the stars. They were known as the Earbay Clan, and they '
            'were led by their wise and gentle leader, Owdgray Earbay. However, one '
            'fateful day, a wicked witch named Ellaay cast a spell on the Earbay Clan, '
            'turning them into ferocious and wild beasts.
           '
            '
           '
            'In the midst of the chaos, a brave and clever erohay named Illylay the '
            'ogfray emerged. Illylay was known for her quick thinking and her love for '
            'her fellow creatures. Determined to save the Earbay Clan, Illylay set out on '
            'a journey to find the ancient and powerful amulet that could break the '
            "witch's spell.
           "
            '
           '
            'With her trusty map and her boundless courage, Illylay hopped from one moon '
            'crater to another, facing 

"Once upon a time, in The Oonmay, there were a group of earsbay who lived peacefully among the stars. They were known as the Earbay Clan, and they were led by their wise and gentle leader, Owdgray Earbay. However, one fateful day, a wicked witch named Ellaay cast a spell on the Earbay Clan, turning them into ferocious and wild beasts.\n\nIn the midst of the chaos, a brave and clever erohay named Illylay the ogfray emerged. Illylay was known for her quick thinking and her love for her fellow creatures. Determined to save the Earbay Clan, Illylay set out on a journey to find the ancient and powerful amulet that could break the witch's spell.\n\nWith her trusty map and her boundless courage, Illylay hopped from one moon crater to another, facing many challenges along the way. She encountered giant space rocks, treacherous lunar storms, and even a mischievous group of moon rabbits. But Illylay never lost hope, for she knew that the fate of the Earbay Clan depended on her bravery.\n\nFinall

We can do the same thing with more structured data, like a CSV

In [123]:
# define the arguments we are goint to use
simple_chat_args = {
    'temperature': 0.8,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

description = "Generate a list of 10 people who work in an office. Include id, name, email address, and salary."

# create the messages we are going to use to create the story.
system_message = {"role":"system", "content":"You are a helpful assistant who generates CSV data for spreadsheets"}
user_message = {"role":"user", "content": description}

# send the information to OpenAI and get back a response
csv_response = simple_chat(messages=[system_message, user_message])
# extract the response from the larger JSON object that was returned
csv = csv_response.choices[0].message.content

ic(csv)

ic| csv: ('id, name, email, salary
         '
          '1, John Smith, john.smith@example.com, $50,000
         '
          '2, Sarah Johnson, sarah.johnson@example.com, $45,000
         '
          '3, Michael Davis, michael.davis@example.com, $55,000
         '
          '4, Jessica Brown, jessica.brown@example.com, $48,000
         '
          '5, David Wilson, david.wilson@example.com, $52,000
         '
          '6, Jennifer Taylor, jennifer.taylor@example.com, $47,000
         '
          '7, Matthew Anderson, matthew.anderson@example.com, $54,000
         '
          '8, Laura Thomas, laura.thomas@example.com, $46,000
         '
          '9, Andrew Clark, andrew.clark@example.com, $51,000
         '
          '10, Emily Martinez, emily.martinez@example.com, $49,000')


'id, name, email, salary\n1, John Smith, john.smith@example.com, $50,000\n2, Sarah Johnson, sarah.johnson@example.com, $45,000\n3, Michael Davis, michael.davis@example.com, $55,000\n4, Jessica Brown, jessica.brown@example.com, $48,000\n5, David Wilson, david.wilson@example.com, $52,000\n6, Jennifer Taylor, jennifer.taylor@example.com, $47,000\n7, Matthew Anderson, matthew.anderson@example.com, $54,000\n8, Laura Thomas, laura.thomas@example.com, $46,000\n9, Andrew Clark, andrew.clark@example.com, $51,000\n10, Emily Martinez, emily.martinez@example.com, $49,000'

## Exercise 4

Add a column to the CSV for each employees hire date and title.
Modify the settings so that you will get the same results every time you run the cell

In [128]:
# define the arguments we are going to use
simple_chat_args = {
    'temperature': 0.0,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

description = "Generate a list of 10 people who work in an office. Include id, name, email address, salary, hire date, and title."

# create the messages we are going to use to create the story.
system_message = {"role":"system", "content":"You are a helpful assistant who generates CSV data for spreadsheets"}
user_message = {"role":"user", "content": description}

# send the information to OpenAI and get back a response
csv_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
# extract the response from the larger JSON object that was returned
csv = csv_response.choices[0].message.content

ic(csv)

ic| csv: ('id,name,email address,salary,hire date,title
         '
          '1,John Doe,johndoe@example.com,50000,2020-01-01,Manager
         '
          '2,Jane Smith,janesmith@example.com,45000,2020-02-15,Assistant Manager
         '
          '3,Michael Johnson,michaeljohnson@example.com,40000,2020-03-10,Accountant
         '
          '4,Sarah Williams,sarahwilliams@example.com,35000,2020-04-05,Receptionist
         '
          '5,David Brown,davidbrown@example.com,30000,2020-05-20,IT Specialist
         '
          '6,Emily Davis,emilydavis@example.com,25000,2020-06-15,Administrative '
          'Assistant
         '
          '7,Robert Wilson,robertwilson@example.com,20000,2020-07-10,Data Entry Clerk
         '
          '8,Lisa Taylor,lisataylor@example.com,15000,2020-08-05,Office Assistant
         '
          '9,Matthew Anderson,matthewanderson@example.com,10000,2020-09-20,Intern
         '
          '10,Amanda Martinez,amandamartinez@example.com,5000,2020-10-15,Trainee')


'id,name,email address,salary,hire date,title\n1,John Doe,johndoe@example.com,50000,2020-01-01,Manager\n2,Jane Smith,janesmith@example.com,45000,2020-02-15,Assistant Manager\n3,Michael Johnson,michaeljohnson@example.com,40000,2020-03-10,Accountant\n4,Sarah Williams,sarahwilliams@example.com,35000,2020-04-05,Receptionist\n5,David Brown,davidbrown@example.com,30000,2020-05-20,IT Specialist\n6,Emily Davis,emilydavis@example.com,25000,2020-06-15,Administrative Assistant\n7,Robert Wilson,robertwilson@example.com,20000,2020-07-10,Data Entry Clerk\n8,Lisa Taylor,lisataylor@example.com,15000,2020-08-05,Office Assistant\n9,Matthew Anderson,matthewanderson@example.com,10000,2020-09-20,Intern\n10,Amanda Martinez,amandamartinez@example.com,5000,2020-10-15,Trainee'